In [8]:
"""
Hyperparameter Optimization Comparison Study
=============================================
Compares different hyperparameter optimization strategies (Grid Search, Random Search, 
Bayesian Optimization, and Genetic Algorithm) on SVM and MLP classifiers using the 
Pima Indians Diabetes dataset.

Author: Data Science Team
Date: 2024
"""

import os
import warnings
from time import perf_counter
from typing import Any, Dict, List, Tuple

import kagglehub
import numpy as np
import pandas as pd
from scipy.stats import loguniform
from sklearn.impute import SimpleImputer
from sklearn.metrics import (
    accuracy_score,
    cohen_kappa_score,
    f1_score,
    make_scorer,
    precision_score,
    recall_score,
    roc_auc_score,
)
from sklearn.model_selection import GridSearchCV, KFold, RandomizedSearchCV, train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn_genetic import GASearchCV
from sklearn_genetic.space import Categorical as GCategorical
from sklearn_genetic.space import Continuous, Integer as GInteger
from skopt import BayesSearchCV
from skopt.space import Categorical, Integer, Real

# Suppress warnings for cleaner output
warnings.filterwarnings('ignore')


# ================================================================================
# CONFIGURATION
# ================================================================================

RANDOM_STATE = 42
TEST_SIZE = 0.3
CV_SPLITS = 5
PRIMARY_METRIC = "f1"

SCORING_METRICS = {
    "accuracy": "accuracy",
    "precision": make_scorer(precision_score, zero_division=0),
    "recall": make_scorer(recall_score, zero_division=0),
    "f1": "f1",
    "roc_auc": "roc_auc",
    "kappa": make_scorer(cohen_kappa_score),
}

# Hyperparameter search configuration
RANDOM_SEARCH_ITERATIONS = 20
BAYESIAN_SEARCH_ITERATIONS = 30
GENETIC_GENERATIONS = 20
GENETIC_POPULATION_SIZE = 30


# ================================================================================
# DATA LOADING AND PREPROCESSING
# ================================================================================

def load_diabetes_dataset() -> pd.DataFrame:
    """
    Download and load the Pima Indians Diabetes dataset from Kaggle.
    
    Returns:
        pd.DataFrame: Loaded dataset with all features and target variable.
    """
    print("📥 Downloading dataset from Kaggle...")
    dataset_path = kagglehub.dataset_download("uciml/pima-indians-diabetes-database")
    csv_path = os.path.join(dataset_path, "diabetes.csv")
    
    df = pd.read_csv(csv_path)
    print(f"✓ Dataset loaded: {df.shape[0]} samples, {df.shape[1]} features\n")
    return df


def prepare_data(df: pd.DataFrame) -> Tuple[np.ndarray, np.ndarray, np.ndarray, np.ndarray]:
    """
    Split and scale the dataset for training and testing.
    
    Args:
        df: Input DataFrame containing features and target.
        
    Returns:
        Tuple of (X_train, X_test, y_train, y_test) as scaled numpy arrays.
    """
    X = df.drop("Outcome", axis=1)
    y = df["Outcome"]
    
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=TEST_SIZE, random_state=RANDOM_STATE, stratify=y
    )
    
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    print(f"📊 Data split:")
    print(f"   Training set: {X_train_scaled.shape[0]} samples")
    print(f"   Test set: {X_test_scaled.shape[0]} samples")
    print(f"   Class distribution (train): {dict(zip(*np.unique(y_train, return_counts=True)))}\n")
    
    return X_train_scaled, X_test_scaled, y_train, y_test


# ================================================================================
# EVALUATION UTILITIES
# ================================================================================

def evaluate_model_on_test_set(estimator, X_test: np.ndarray, y_test: np.ndarray) -> Dict[str, float]:
    """
    Evaluate a trained model on the test set with multiple metrics.
    
    Args:
        estimator: Trained model estimator.
        X_test: Test features.
        y_test: Test labels.
        
    Returns:
        Dictionary containing all evaluation metrics.
    """
    y_pred = estimator.predict(X_test)
    
    # Get probability estimates for ROC AUC
    if hasattr(estimator, "predict_proba"):
        y_proba = estimator.predict_proba(X_test)[:, 1]
    elif hasattr(estimator, "decision_function"):
        y_proba = estimator.decision_function(X_test)
    else:
        y_proba = None
    
    metrics = {
        "accuracy": accuracy_score(y_test, y_pred),
        "precision": precision_score(y_test, y_pred, zero_division=0),
        "recall": recall_score(y_test, y_pred, zero_division=0),
        "f1": f1_score(y_test, y_pred),
        "kappa": cohen_kappa_score(y_test, y_pred),
    }
    
    if y_proba is not None:
        metrics["roc_auc"] = roc_auc_score(y_test, y_proba)
    else:
        metrics["roc_auc"] = np.nan
    
    return metrics


def extract_cv_results(search_object) -> Dict[str, float]:
    """
    Extract cross-validation results from a search object.
    
    Args:
        search_object: Fitted hyperparameter search object.
        
    Returns:
        Dictionary containing CV metrics (mean and std).
    """
    best_idx = getattr(search_object, "best_index_", None)
    cv_results = {}
    
    # GASearchCV and BayesSearchCV only have mean_test_score and std_test_score
    if hasattr(search_object, "cv_results_") and "mean_test_score" in search_object.cv_results_:
        if best_idx is None:
            best_idx = 0
        cv_results["cv_f1_mean"] = search_object.cv_results_["mean_test_score"][best_idx]
        cv_results["cv_f1_std"] = search_object.cv_results_["std_test_score"][best_idx]
        return cv_results
    
    # For GridSearchCV and RandomizedSearchCV, extract all metrics
    for metric_name in SCORING_METRICS.keys():
        try:
            cv_results[f"cv_{metric_name}_mean"] = search_object.cv_results_[f"mean_test_{metric_name}"][best_idx]
            cv_results[f"cv_{metric_name}_std"] = search_object.cv_results_[f"std_test_{metric_name}"][best_idx]
        except (KeyError, TypeError):
            pass
    
    return cv_results


# ================================================================================
# HYPERPARAMETER SEARCH SPACES
# ================================================================================

def get_svm_search_spaces() -> Dict[str, Any]:
    """
    Define hyperparameter search spaces for SVM classifier.
    
    Returns:
        Dictionary containing search spaces for each optimization strategy.
    """
    grid_space = [
        {
            "clf__kernel": ["linear"],
            "clf__C": [1e-3, 1e-2, 1e-1, 1, 10, 1e2, 1e3],
            "clf__class_weight": [None, "balanced"],
        },
        {
            "clf__kernel": ["rbf"],
            "clf__C": [1e-3, 1e-2, 1e-1, 1, 10, 1e2, 1e3],
            "clf__gamma": [1e-4, 1e-3, 1e-2, 1e-1, 1.0],
            "clf__class_weight": [None, "balanced"],
        },
    ]
    
    random_space = {
        "clf__kernel": ["linear", "rbf"],
        "clf__C": loguniform(1e-3, 1e3),
        "clf__gamma": loguniform(1e-4, 1.0),
        "clf__class_weight": [None, "balanced"],
    }
    
    # Bayesian optimization: single dict (gamma ignored when kernel='linear')
    bayesian_space = {
        "clf__kernel": Categorical(["linear", "rbf"]),
        "clf__C": Real(1e-3, 1e3, prior="log-uniform"),
        "clf__gamma": Real(1e-4, 1.0, prior="log-uniform"),
        "clf__class_weight": Categorical([None, "balanced"]),
    }
    
    genetic_space = {
        "clf__kernel": GCategorical(["linear", "rbf"]),
        "clf__C": Continuous(1e-3, 1e3, distribution="log-uniform"),
        "clf__gamma": Continuous(1e-4, 1.0, distribution="log-uniform"),
        "clf__class_weight": GCategorical([None, "balanced"]),
    }
    
    return {
        "grid": grid_space,
        "random": random_space,
        "bayesian": bayesian_space,
        "genetic": genetic_space,
    }


def get_mlp_search_spaces() -> Dict[str, Any]:
    """
    Define hyperparameter search spaces for MLP classifier.
    
    Returns:
        Dictionary containing search spaces for each optimization strategy.
    """
    hidden_layer_options = [(64,), (128,), (64, 32), (128, 64), (256, 128)]
    
    grid_space = {
        "clf__hidden_layer_sizes": hidden_layer_options,
        "clf__activation": ["relu", "tanh"],
        "clf__solver": ["adam", "lbfgs"],
        "clf__alpha": [1e-6, 1e-5, 1e-4, 1e-3, 1e-2],
        "clf__learning_rate_init": [1e-4, 1e-3, 1e-2],
        "clf__max_iter": [300, 600],
        "clf__early_stopping": [True],
    }
    
    random_space = {
        "clf__hidden_layer_sizes": hidden_layer_options,
        "clf__activation": ["relu", "tanh"],
        "clf__solver": ["adam", "lbfgs"],
        "clf__alpha": loguniform(1e-6, 1e-2),
        "clf__learning_rate_init": loguniform(1e-4, 1e-2),
        "clf__max_iter": [300, 600],
        "clf__early_stopping": [True],
    }
    
    bayesian_space = {
        "clf__hidden_layer_sizes": Categorical(hidden_layer_options),
        "clf__activation": Categorical(["relu", "tanh"]),
        "clf__solver": Categorical(["adam", "lbfgs"]),
        "clf__alpha": Real(1e-6, 1e-2, prior="log-uniform"),
        "clf__learning_rate_init": Real(1e-4, 1e-2, prior="log-uniform"),
        "clf__max_iter": Integer(300, 600),
        "clf__early_stopping": Categorical([True]),
    }
    
    genetic_space = {
        "clf__hidden_layer_sizes": GCategorical(hidden_layer_options),
        "clf__activation": GCategorical(["relu", "tanh"]),
        "clf__solver": GCategorical(["adam", "lbfgs"]),
        "clf__alpha": Continuous(1e-6, 1e-2, distribution="log-uniform"),
        "clf__learning_rate_init": Continuous(1e-4, 1e-2, distribution="log-uniform"),
        "clf__max_iter": GInteger(300, 600),
        "clf__early_stopping": GCategorical([True]),
    }
    
    return {
        "grid": grid_space,
        "random": random_space,
        "bayesian": bayesian_space,
        "genetic": genetic_space,
    }


# ================================================================================
# HYPERPARAMETER OPTIMIZATION
# ================================================================================

def run_hyperparameter_search(
    model_name: str,
    pipeline: Pipeline,
    search_spaces: Dict[str, Any],
    X_train: np.ndarray,
    y_train: np.ndarray,
    X_test: np.ndarray,
    y_test: np.ndarray,
    cv: KFold,
) -> List[Dict[str, Any]]:
    """
    Run all hyperparameter optimization strategies for a given model.
    
    Args:
        model_name: Name of the model being optimized.
        pipeline: sklearn Pipeline with preprocessing and classifier.
        search_spaces: Dictionary of search spaces for each strategy.
        X_train: Training features.
        y_train: Training labels.
        X_test: Test features.
        y_test: Test labels.
        cv: Cross-validation splitter.
        
    Returns:
        List of dictionaries containing results for each optimization strategy.
    """
    results = []
    
    print(f"\n{'='*80}")
    print(f"HYPERPARAMETER OPTIMIZATION: {model_name}")
    print(f"{'='*80}")
    
    # ============================================================================
    # 1. GRID SEARCH
    # ============================================================================
    print(f"\n[1/4] Running Grid Search...")
    start_time = perf_counter()
    
    try:
        grid_search = GridSearchCV(
            estimator=pipeline,
            param_grid=search_spaces["grid"],
            cv=cv,
            scoring=SCORING_METRICS,
            refit=PRIMARY_METRIC,
            n_jobs=-1,
            verbose=0,
        )
        grid_search.fit(X_train, y_train)
        elapsed_time = perf_counter() - start_time
        
        result = {
            "model": model_name,
            "strategy": "Grid Search",
            "time_s": elapsed_time,
            "best_params": grid_search.best_params_,
        }
        result.update(extract_cv_results(grid_search))
        result.update({f"test_{k}": v for k, v in evaluate_model_on_test_set(grid_search.best_estimator_, X_test, y_test).items()})
        results.append(result)
        print(f"   ✓ Completed in {elapsed_time:.2f}s | Best CV F1: {result.get('cv_f1_mean', 0):.4f}")
    except Exception as e:
        print(f"   ✗ Failed: {str(e)}")
    
    # ============================================================================
    # 2. RANDOM SEARCH
    # ============================================================================
    print(f"\n[2/4] Running Random Search...")
    start_time = perf_counter()
    
    try:
        random_search = RandomizedSearchCV(
            estimator=pipeline,
            param_distributions=search_spaces["random"],
            n_iter=RANDOM_SEARCH_ITERATIONS,
            cv=cv,
            scoring=SCORING_METRICS,
            refit=PRIMARY_METRIC,
            n_jobs=-1,
            random_state=RANDOM_STATE,
            verbose=0,
        )
        random_search.fit(X_train, y_train)
        elapsed_time = perf_counter() - start_time
        
        result = {
            "model": model_name,
            "strategy": "Random Search",
            "time_s": elapsed_time,
            "best_params": random_search.best_params_,
        }
        result.update(extract_cv_results(random_search))
        result.update({f"test_{k}": v for k, v in evaluate_model_on_test_set(random_search.best_estimator_, X_test, y_test).items()})
        results.append(result)
        print(f"   ✓ Completed in {elapsed_time:.2f}s | Best CV F1: {result.get('cv_f1_mean', 0):.4f}")
    except Exception as e:
        print(f"   ✗ Failed: {str(e)}")
    
    # ============================================================================
    # 3. BAYESIAN OPTIMIZATION
    # ============================================================================
    print(f"\n[3/4] Running Bayesian Optimization...")
    start_time = perf_counter()
    
    try:
        bayesian_search = BayesSearchCV(
            estimator=pipeline,
            search_spaces=search_spaces["bayesian"],
            n_iter=BAYESIAN_SEARCH_ITERATIONS,
            cv=cv,
            scoring=make_scorer(f1_score),  # Single metric for BayesSearchCV
            n_jobs=-1,
            random_state=RANDOM_STATE,
            verbose=0,
        )
        bayesian_search.fit(X_train, y_train)
        elapsed_time = perf_counter() - start_time
        
        result = {
            "model": model_name,
            "strategy": "Bayesian Optimization",
            "time_s": elapsed_time,
            "best_params": bayesian_search.best_params_,
        }
        result.update(extract_cv_results(bayesian_search))
        result.update({f"test_{k}": v for k, v in evaluate_model_on_test_set(bayesian_search.best_estimator_, X_test, y_test).items()})
        results.append(result)
        print(f"   ✓ Completed in {elapsed_time:.2f}s | Best CV F1: {result.get('cv_f1_mean', 0):.4f}")
    except Exception as e:
        print(f"   ✗ Failed: {str(e)}")
    
    # ============================================================================
    # 4. GENETIC ALGORITHM
    # ============================================================================
    print(f"\n[4/4] Running Genetic Algorithm...")
    start_time = perf_counter()
    
    try:
        genetic_search = GASearchCV(
            estimator=pipeline,
            cv=cv,
            scoring=make_scorer(f1_score),
            n_jobs=-1,
            param_grid=search_spaces["genetic"],
            generations=GENETIC_GENERATIONS,
            population_size=GENETIC_POPULATION_SIZE,
            tournament_size=3,
            elitism=True,
            mutation_probability=0.1,
            crossover_probability=0.8,
            verbose=False,
        )
        genetic_search.fit(X_train, y_train)
        elapsed_time = perf_counter() - start_time
        
        result = {
            "model": model_name,
            "strategy": "Genetic Algorithm",
            "time_s": elapsed_time,
            "best_params": genetic_search.best_params_,
        }
        result.update(extract_cv_results(genetic_search))
        result.update({f"test_{k}": v for k, v in evaluate_model_on_test_set(genetic_search.best_estimator_, X_test, y_test).items()})
        results.append(result)
        print(f"   ✓ Completed in {elapsed_time:.2f}s | Best CV F1: {result.get('cv_f1_mean', 0):.4f}")
    except Exception as e:
        print(f"   ✗ Failed: {str(e)}")
    
    return results


# ================================================================================
# RESULTS PRESENTATION
# ================================================================================

def display_results_summary(results_df: pd.DataFrame) -> None:
    """
    Display a comprehensive summary of optimization results.
    
    Args:
        results_df: DataFrame containing all optimization results.
    """
    print("\n" + "="*100)
    print("HYPERPARAMETER OPTIMIZATION RESULTS SUMMARY")
    print("="*100)
    
    # Overall best model
    best_idx = results_df["test_f1"].idxmax()
    best_result = results_df.loc[best_idx]
    
    print(f"\n🏆 BEST OVERALL CONFIGURATION:")
    print(f"   Model: {best_result['model']}")
    print(f"   Strategy: {best_result['strategy']}")
    print(f"   Test F1 Score: {best_result['test_f1']:.4f}")
    print(f"   Test Accuracy: {best_result['test_accuracy']:.4f}")
    print(f"   Test ROC-AUC: {best_result['test_roc_auc']:.4f}")
    print(f"   Optimization Time: {best_result['time_s']:.2f}s")
    print(f"   Best Parameters: {best_result['best_params']}")
    
    # Summary by model
    print(f"\n{'─'*100}")
    print("RESULTS BY MODEL:")
    print(f"{'─'*100}")
    
    for model_name in results_df["model"].unique():
        model_results = results_df[results_df["model"] == model_name]
        print(f"\n{model_name}:")
        
        for _, row in model_results.iterrows():
            print(f"  • {row['strategy']:25s} | F1: {row['test_f1']:.4f} | "
                  f"Accuracy: {row['test_accuracy']:.4f} | "
                  f"ROC-AUC: {row['test_roc_auc']:.4f} | "
                  f"Time: {row['time_s']:7.2f}s")
    
    # Performance comparison
    print(f"\n{'─'*100}")
    print("STRATEGY PERFORMANCE COMPARISON (Average across models):")
    print(f"{'─'*100}")
    
    strategy_summary = results_df.groupby("strategy").agg({
        "test_f1": ["mean", "std"],
        "test_accuracy": ["mean", "std"],
        "time_s": ["mean", "std"]
    }).round(4)
    
    for strategy in strategy_summary.index:
        f1_mean = strategy_summary.loc[strategy, ("test_f1", "mean")]
        f1_std = strategy_summary.loc[strategy, ("test_f1", "std")]
        acc_mean = strategy_summary.loc[strategy, ("test_accuracy", "mean")]
        time_mean = strategy_summary.loc[strategy, ("time_s", "mean")]
        
        print(f"  {strategy:25s} | F1: {f1_mean:.4f} (±{f1_std:.4f}) | "
              f"Acc: {acc_mean:.4f} | Avg Time: {time_mean:7.2f}s")
    
    print(f"\n{'='*100}\n")


def save_detailed_results(results_df: pd.DataFrame, output_path: str = "optimization_results.csv") -> None:
    """
    Save detailed results to a CSV file.
    
    Args:
        results_df: DataFrame containing all results.
        output_path: Path where to save the CSV file.
    """
    results_df.to_csv(output_path, index=False)
    print(f"📊 Detailed results saved to: {output_path}")


def display_detailed_table(results_df: pd.DataFrame) -> None:
    """
    Display detailed results table with formatting.
    
    Args:
        results_df: DataFrame containing all results.
    """
    print("\nDETAILED RESULTS TABLE:")
    print("="*100)
    
    # Configure pandas display options
    pd.set_option('display.max_columns', None)
    pd.set_option('display.width', None)
    pd.set_option('display.max_colwidth', 50)
    pd.set_option('display.precision', 4)
    
    # Select and order columns for display
    display_cols = [
        "model", "strategy", "time_s",
        "cv_f1_mean", "cv_f1_std",
        "test_f1", "test_accuracy", "test_precision", "test_recall",
        "test_roc_auc", "test_kappa"
    ]
    display_cols = [col for col in display_cols if col in results_df.columns]
    
    print(results_df[display_cols].to_string(index=False))
    print("="*100)


# ================================================================================
# MAIN EXECUTION
# ================================================================================

def main():
    """Main execution function for the hyperparameter optimization study."""
    print("\n" + "="*100)
    print("HYPERPARAMETER OPTIMIZATION COMPARISON STUDY")
    print("Comparing Grid Search, Random Search, Bayesian Optimization, and Genetic Algorithms")
    print("="*100)
    
    # Load and prepare data
    df = load_diabetes_dataset()
    X_train, X_test, y_train, y_test = prepare_data(df)
    
    # Setup cross-validation
    cv = KFold(n_splits=CV_SPLITS, shuffle=True, random_state=RANDOM_STATE)
    print(f"🔄 Cross-validation: {CV_SPLITS}-fold stratified\n")
    
    all_results = []
    
    # ============================================================================
    # SVM OPTIMIZATION
    # ============================================================================
    print(f"\n{'#'*80}")
    print("# PART 1: SUPPORT VECTOR MACHINE (SVM) OPTIMIZATION")
    print(f"{'#'*80}")
    
    svm_pipeline = Pipeline(steps=[
        ("imputer", SimpleImputer(strategy="median")),
        ("scaler", StandardScaler()),
        ("clf", SVC(probability=True, random_state=RANDOM_STATE))
    ])
    
    svm_spaces = get_svm_search_spaces()
    svm_results = run_hyperparameter_search(
        model_name="SVM",
        pipeline=svm_pipeline,
        search_spaces=svm_spaces,
        X_train=X_train,
        y_train=y_train,
        X_test=X_test,
        y_test=y_test,
        cv=cv,
    )
    all_results.extend(svm_results)
    
    # ============================================================================
    # MLP OPTIMIZATION
    # ============================================================================
    print(f"\n{'#'*80}")
    print("# PART 2: MULTI-LAYER PERCEPTRON (MLP) OPTIMIZATION")
    print(f"{'#'*80}")
    
    mlp_pipeline = Pipeline(steps=[
        ("imputer", SimpleImputer(strategy="median")),
        ("scaler", StandardScaler()),
        ("clf", MLPClassifier(random_state=RANDOM_STATE))
    ])
    
    mlp_spaces = get_mlp_search_spaces()
    mlp_results = run_hyperparameter_search(
        model_name="MLP",
        pipeline=mlp_pipeline,
        search_spaces=mlp_spaces,
        X_train=X_train,
        y_train=y_train,
        X_test=X_test,
        y_test=y_test,
        cv=cv,
    )
    all_results.extend(mlp_results)
    
    # ============================================================================
    # RESULTS ANALYSIS AND VISUALIZATION
    # ============================================================================
    print(f"\n{'#'*80}")
    print("# RESULTS ANALYSIS")
    print(f"{'#'*80}")
    
    results_df = pd.DataFrame(all_results)
    
    # Reorder columns for better readability
    column_order = [
        "model", "strategy", "time_s", "best_params",
        "cv_f1_mean", "cv_f1_std",
        "test_f1", "test_accuracy", "test_precision", "test_recall",
        "test_roc_auc", "test_kappa"
    ]
    column_order = [col for col in column_order if col in results_df.columns]
    other_columns = [col for col in results_df.columns if col not in column_order]
    results_df = results_df[column_order + other_columns]
    
    # Display comprehensive summary
    display_results_summary(results_df)
    
    # Display detailed table
    display_detailed_table(results_df)
    
    # Save results
    save_detailed_results(results_df)
    
    print("\n✅ Hyperparameter optimization study completed successfully!")
    print(f"   Total optimization time: {results_df['time_s'].sum():.2f}s")
    print(f"   Number of configurations tested: {len(results_df)}\n")


if __name__ == "__main__":
    main()


HYPERPARAMETER OPTIMIZATION COMPARISON STUDY
Comparing Grid Search, Random Search, Bayesian Optimization, and Genetic Algorithms
📥 Downloading dataset from Kaggle...
✓ Dataset loaded: 768 samples, 9 features

📊 Data split:
   Training set: 537 samples
   Test set: 231 samples
   Class distribution (train): {np.int64(0): np.int64(350), np.int64(1): np.int64(187)}

🔄 Cross-validation: 5-fold stratified


################################################################################
# PART 1: SUPPORT VECTOR MACHINE (SVM) OPTIMIZATION
################################################################################

HYPERPARAMETER OPTIMIZATION: SVM

[1/4] Running Grid Search...
   ✓ Completed in 19.69s | Best CV F1: 0.6621

[2/4] Running Random Search...
   ✓ Completed in 0.99s | Best CV F1: 0.6567

[3/4] Running Bayesian Optimization...
   ✓ Completed in 88.85s | Best CV F1: 0.6631

[4/4] Running Genetic Algorithm...
   ✓ Completed in 163.39s | Best CV F1: 0.6599

######################